**# Importanto o Dataset e verificando o conteúdo**

In [1]:
import sys
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') #carrego stopwrds para filtragem NLP

# Adiciona o caminho da pasta onde o arquivo 'dataset.py' está localizado
# Isso pula a necessidade de mencionar a pasta com hífen no import
caminho_raw = os.path.abspath(os.path.join('..', 'data', 'raw'))
if caminho_raw not in sys.path:
    sys.path.append(caminho_raw)

from dataset import carregar_dados

df = carregar_dados()
print(df.head())

[nltk_data] Downloading package stopwords to /home/rafael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                           issue_url  \
0  "https://github.com/zhangyuanwei/node-images/i...   
1     "https://github.com/Microsoft/pxt/issues/2543"   
2  "https://github.com/MatisiekPL/Czekolada/issue...   
3  "https://github.com/MatisiekPL/Czekolada/issue...   
4  "https://github.com/MatisiekPL/Czekolada/issue...   

                                         issue_title  \
0  can't load the addon. issue to: https://github...   
1  hcl accessibility a11yblocking a11ymas mas4.2....   
2  issue 1265: issue 1264: issue 1261: issue 1260...   
3  issue 1266: issue 1263: issue 1262: issue 1259...   
4  issue 1288: issue 1285: issue 1284: issue 1281...   

                                                body  
0  can't load the addon. issue to: https://github...  
1  user experience: user who depends on screen re...  
2  ┆attachments: <a href= https:& x2f;& x2f;githu...  
3  gitlo = github x trello\n---\nthis board is no...  
4  ┆attachments: <a href= https:& x2f;& x2f;githu..

/home/rafael/Desktop/Portifolio/Intelligent-Assistant-for-Data-and-Document-Analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/rafael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Processamento e limpeza dos dados**


In [2]:
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'raw')))
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'processed')))


from dataset import verifica_vazios, verifica_frequentes
from data_processing import clean_text

df["clean_title"] = df["issue_title"].apply(clean_text)
df["clean_body"] = df["body"].apply(clean_text)


media_palavras_url = df["issue_url"].str.split().str.len().mean()
media_palavras_title = df["clean_title"].str.split().str.len().mean()
media_palavras_body = df["clean_body"].str.split().str.len().mean()

print(f"Média de palavras em url: {media_palavras_url}, em title: {media_palavras_title}, em body: {media_palavras_body}")

url_vazio = verifica_vazios(df,"issue_url")
titles_vazio = verifica_vazios(df,"clean_title")
body_vazio = verifica_vazios(df,"clean_body")

palavras_frequentes_titles = verifica_frequentes(df,"clean_title", 20)
print(f"Palavras mais frequentes em titles: {palavras_frequentes_titles}")
palavras_frequentes_body = verifica_frequentes(df,"clean_body", 20)
print(f"Palavras mais frequentes em body: {palavras_frequentes_body}")



Média de palavras em url: 1.0, em title: 10.9025, em body: 38.63875
Resultados para 'issue_url':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 0
Resultados para 'clean_title':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 69
Resultados para 'clean_body':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 4
Palavras mais frequentes em titles: [('issu', 352), ('add', 35), ('error', 21), ('bug', 21), ('use', 18), ('set', 12), ('test', 10), ('line', 9), ('task', 9), ('jquery', 8), ('api', 8), ('fix', 6), ('training', 6), ('network', 6), ('app', 5), ('help', 5), ('npm', 5), ('load', 4), ('button', 4), ('warning', 4)]
Palavras mais frequentes em body: [('close', 1237), ('add', 939), ('issues', 898), ('columns', 704), ('list', 540), ('comment', 538), ('column', 531), ('trello', 528), ('update', 367), ('via', 360), ('default', 360), ('move', 358), ('settings', 357), ('custom', 355), ('attachments', 352), ('matisiekpl', 352), ('czekolada', 352), ('gitlo', 352), ('board', 352), ('sync', 3


   **Análise descritiva dos dados processados**

- Média de palavras para entender quantos tokens irão ser processados pela LLM
    - Quantidade de células vazias em todas as colunas
        - não teve resultado de células vazias, todas preenchidas
    - Títulos - tamanhos médio de 5,4 palavras
    - Body - Aproximadamente 28 palavras
- Ideal para:
    - Embeddings
    - Chunking Leve
    - RAG Eficiente(baixo custo e boa semântica)

**Criação de Texto final**

In [3]:
from sentence_transformers import SentenceTransformer

#criação de nova coluna para texto final que será direcionado ao embedding
df["final_text"] = (
    "Title: " + df["clean_title"] +
    ". Body: " + df["clean_body"]
)

#carregando o modelo que será usado, modelo rápido e leve para projeto OBS: Uso da CPU pois GPU esta ultrapassada para o modelo
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device="cpu") #usando o paraphrasal para poder perguntar em portugues
df[["clean_title", "clean_body", "final_text"]].head()

,clean_title,clean_body,final_text
0,load addon issue error lib libc version glibc ...,load addon issue error lib libc version glibc ...,Title: load addon issue error lib libc version...
1,hcl accessibility yblocking ymas mas hcl makec...,user experience user depends screen reader get...,Title: hcl accessibility yblocking ymas mas hc...
2,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...
3,issue issue issue issue issue issue issue issu...,gitlo github trello board linked update issue ...,Title: issue issue issue issue issue issue iss...
4,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...


**Embedding**

In [4]:
import numpy as np

#listando o texto final em variavel para ser codificada
texts = df["final_text"].tolist()

#realização do embedding pelo modelo escolhido
embeddings = model.encode( 
    texts,
    show_progress_bar= True
)

#criação do array em np 
embeddings = np.array(embeddings)
print(embeddings.shape)

#persistindo os dados em formato npy e csv para nao necessitar de conversao novamente
np.save("embeddings.npy",embeddings)
df.to_csv("issue_processed.csv", index=False)


Batches: 100%|██████████| 25/25 [00:13<00:00,  1.79it/s]

(800, 384)


**Busca Semântica**

- Primeiramente utilizarei busca ingênua para este caso, mais rápido e ideal para projetos pequenos.
    - será utilizado semelhança de cossenos, variância de -1 a 1, sendo 1 o mais próximo.
    - comparação de vetores do embedding, modelo utilizado de dimensão 384
- Após implementado e projeto funconando, irei dar updgrade para utilizar índice vetorial
- Devido a arquitetura proposta relacionado aos tipos de perguntas para o Chatbot, será implementado:
    - Busca semântica
    - Top k
    - Limiar de similiaridade
    Dessa forma será possível a resposta de perguntas analtícas e explicativas, não somente localizadoras.

In [5]:
import pandas as pd #ler diretamente esta célula sem precisar converter novamente os arquivos no embedding
import numpy as np

embeddings = np.load("embeddings.npy")
df = pd.read_csv("issue_processed.csv")

**Exemplo de pergunta e formato para query**

In [6]:
from semantic_search import encode_query, cosine_similiarity_func

pergunta = "Quais sao os erros mais registrados no documento?"
query = encode_query(model, [pergunta])
#print(query.shape)

scores = cosine_similiarity_func(query,embeddings)
scores = scores.flatten()
scores_ordenados = np.argsort(scores)
print(scores.shape)
print(scores_ordenados)

#fazendo busca top_k, depois refatorar montando em funcoes definidas
# Atualizar o Readme e usar o topk como parametro para busca ao refatorar “Utilizamos top-k dinâmico para balancear cobertura semântica e precisão.”
top_k = 30
top_indices = scores_ordenados[-top_k:][::-1] #quero buscar os ultimos 20 valores e ordena-los em sequência maior para menor, pois np.arg retorna ordem crescente
print(top_indices)




(800,)
[697 496 603 745 622 499 500 564 703 665 576 534 785 640 459 789 470 482
 546 474 537 469 478 788 402 460 518 517 710 467 625 560 498 610 452 720
 398 621 747 731 431 704 727 793 435 357 646 420 696 392 511 570 598 716
 529 449 779 515 520 464 626 602 664 508 709 584 512 648 481 714 561 740
 495 632 591 623 624 590 551 365 391 516 634 773 367 397 780 422 577   1
 404 530 531 362 566 796 668 699 477 439 501 458 463 627 631 382 361 616
 519 456 644 413 438 712 609 690 368 521 409 542 765 600 383 642 790 574
 387 514 707 550 376 480 629 562 505 506 630 657 388 728 746 509 430 782
 633 615 389 523 651 381 504 484 372 354 472 702 713 679 638 726 555 502
 799 488 457 370 647 769 652 721 676 667 685 733 628 669 674 582 533 675
 355 573 766 619 578 794 487 620 741 777 744 694 507 586 743 541 489 661
 408 599 604   0 407 725 594 406 605 543 447 705 639 468 756 763 760 757
 759 758 762 761 471 559 768 798 792 485 589 593 553 379 706 532 426 371
 395 524 423 770 579 781 437 695 358 742 737

**Implementando Limiar de similaridade**

 - Utilizei um limiar de similaridade cosseno ajustado empiricamente para garantir que apenas documentos semanticamente relevantes sejam utilizados como contexto para a LLM, reduzindo ruído e alucinação.
 - A princípio foi definido padrão = 0.30, após verifiquei de forma empirica os melhores valores para retornar respostas completas sem interferências de dados fora do padrão

In [7]:
#definir o limiar, fazer um loop retornando os melhores índices
limiar = 0.30
lista_final = []

for i in range(len(top_indices)):
    if scores[top_indices[i]] > limiar:
        lista_final.append({"indice" : top_indices[i] ,"score" :scores[top_indices[i]], "text" : df.iloc[top_indices[i]]["final_text"]})

print(lista_final)

[{'indice': np.int64(571), 'score': np.float32(0.4546029), 'text': 'Title: . Body: according demo code returns standard errors according code column naming seems return standard deviations meantime assuming model returns standard deviations using boot boot empirical standard errors slow already including less unwieldy method computing kind standard errors report paper would happy clarification also information getting code work kept running error error abs max beta beta tol maxit missing value true false needed ultimately got around explicitly setting intercept input formula model matrix computed model matrix zero'}, {'indice': np.int64(736), 'score': np.float32(0.42092642), 'text': 'Title: null. Body: bulk insert seemed successful according log however checked database data inserted might cause book import books'}, {'indice': np.int64(539), 'score': np.float32(0.41807178), 'text': 'Title: insert. Body: organization name individual dcma organization type document reference include sect

**Construção do contexto**

- Utilizar dos dados retornados a partir do limiar e construir os textos mais similares para servir de entrada para LLM
- estou retornando em texto todos os documentos analisados que passaram pelo limiar, poderia restringir a quantidade caso o número de tokens a ser utilizado na LLM seja limitado

In [8]:
MAX_CHARS = 800  # por documento

lista_final = sorted(
    lista_final,
    key=lambda x: x["score"],
    reverse=True
)

contexto = ""
limite = 28

if len(lista_final) <= limite:
    for i in range(len(lista_final)):
        text_limited = lista_final[i]["text"][:MAX_CHARS]

        contexto += (
            f"[Contexto {i+1} | "
            f"Similaridade: {lista_final[i]['score']:.03f}]\n"
            f"{text_limited}\n\n"
        )

else:
    for i in range(limite):
        text_limited = lista_final[i]["text"][:MAX_CHARS]

        contexto += (
            f"[Contexto {i+1} | "
            f"Similaridade: {lista_final[i]['score']:.03f}]\n"
            f"{text_limited}\n\n"
        )

print(contexto)

[Contexto 1 | Similaridade: 0.455]
Title: . Body: according demo code returns standard errors according code column naming seems return standard deviations meantime assuming model returns standard deviations using boot boot empirical standard errors slow already including less unwieldy method computing kind standard errors report paper would happy clarification also information getting code work kept running error error abs max beta beta tol maxit missing value true false needed ultimately got around explicitly setting intercept input formula model matrix computed model matrix zero

[Contexto 2 | Similaridade: 0.421]
Title: null. Body: bulk insert seemed successful according log however checked database data inserted might cause book import books

[Contexto 3 | Similaridade: 0.418]
Title: insert. Body: organization name individual dcma organization type document reference include section paragraph number executive summary comment include rationale comment insert suggested change read r

**Definir tipos de perguntas para ajustar prompt e resposta**

- Dividir as perguntas em 3 tipos
    - (A) Podem ser respondidas claramente pelo modelo. Ex: "quais tipos", "quais problemas", "sobre o que", "resuma"..
    - (B) Podem ser respondidas com ressalvas. Ex: “mais comuns”,“mais registrados”,“principais” ..
    - (C) O Modelo pode alucinar e dar estatísticas erradas, não recomendado para o propósito da IA. Ex: "quantos", "porcentagem", "frequência", "exata quantidade" ..

In [9]:
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'prompts')))
from rag_prompts import classify_question, build_direct_prompt, build_qualitative_prompt, build_out_of_scope_prompt, build_prompt

QUESTION_TYPE_A = "DIRECT"
QUESTION_TYPE_B = "QUALITATIVE" 
QUESTION_TYPE_C = "OUT_OF_SCOPE"

question_type = classify_question(pergunta)
print("Tipo da pergunta:", question_type)


Tipo da pergunta: QUALITATIVE


**Validador de prompts**

- Antes de passar para LLM analisar, o validador irá verificar se a pergunta se enquadra num tópico a ser analisado ou pode ser respondido sem LLM
- Irá atuar como Gatekeeper, reduzindo a quantidade de tokens analisados
- Evita desperdício de Quota da LLM e redução de custo

**Integração da LLM**

- Estou usando Gemini devido a facilidade de obtenção de uma chave gratuita para estudantes
- A LLM irá receber as informações retiradas do Dataset 
- Será utilizado engenharia de prompt para qualificar a análise da LLM
- No projeto será implantado um scan de input para o usuário digitar sua propria chave api gemini e testar o programa


In [13]:
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'llm')))

from context_validator import validate_context, static_fallback, insufficient_context_response
from dotenv import load_dotenv
import os
from rag_prompts import build_prompt
from gemini_client import generate_answer, answer_question


load_dotenv()
chave_api = os.getenv("GOOGLE_API_KEY")
resposta = answer_question(pergunta,contexto)
print(resposta)

Existem múltiplos relatos de "crashes" ou falhas, frequentemente associados a dispositivos e mencionados com informações de "crashlytics".

Também se observam menções a erros relacionados a "insert" (inserção), com detalhes sobre dados a serem incluídos como nomes de organização e tipos.

Há também menções de erros de "bug" e "issue", algumas vezes repetidas com a palavra "number", sugerindo que se referem a identificadores de problemas.

Outro padrão recorrente são erros relacionados a cálculo ou retorno de valores, como "standard errors", "standard deviations", "standard deviations", "beta", e "entero decimal".

Há também erros de compilação ou de lógica em código, como problemas com "lua function", "compilechunk", "parser rule", e "plural expression".

Além disso, há relatos de dados inválidos ou problemas de validação, como "invalid", "parser rule nplurals integer plural expression", e "form validations registrations".
